### Import Required Libraries and Set Up Environment Variables

In [30]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [31]:
# Set environment variables from the .env in the local environment
load_dotenv('api_keys.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [32]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"


# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

response_movie_reviews = requests.get(query_url).json()
print(json.dumps(response_movie_reviews, indent=4))



{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [33]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):

    # create query with a page number
    # API results show 10 articles at a time
    page_number = f'&page={page}'
    query_url = query_url + page_number
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    
    # Try and save the reviews to the reviews_list
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

            
        # Print the page that was just retrieved
        print(f'Checked page {page}')

        
    except:
        # Print the page number that had no results then break from the loop
        print(f"No results for page: {page}.")
        #break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [34]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(json.dumps(reviews_list[i], indent=4))


{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [35]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_reviews_df = pd.json_normalize(reviews_list)
nyt_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
196,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
197,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
198,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None


In [36]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

# Extract the title from the "headline.main" column and save it to a new column "title"
nyt_reviews_df['title'] = nyt_reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
nyt_reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [37]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
# 'apply()' is used to apply the extract_keywords function to each cell in the "keywords" column.
# lambda to check if x is a list before applying the extract_keywords function
nyt_reviews_df['keywords'] = nyt_reviews_df['keywords'].apply(lambda x: extract_keywords(x) if isinstance(x, list) else "")
nyt_reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [38]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = nyt_reviews_df['title'].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide

### Access The Movie Database API

In [39]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [40]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)


    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    my_url = (f'https://api.themoviedb.org/3/search/movie?query={title}&api_key={tmdb_api_key}')
    response_tmdb_search = requests.get(my_url).json()
    #print(json.dumps(response_tmdb_search, indent=4))


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:

        # Get movie id
        movie_id = response_tmdb_search["results"][0]["id"]
        #print(movie_id)
        

        # Make a request for a the full movie details
        tmdb_query_for_details = (f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}')


        # Execute "GET" request with url
        response_tmdb_query_details = requests.get(tmdb_query_for_details).json()
        #print(json.dumps(response_tmdb_query_details, indent=4))

        
        # Extract the genre names into a list
        genres = [genre["name"] for genre in response_tmdb_query_details["genres"]]
        #print(genres)


        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in response_tmdb_query_details['spoken_languages']]
        #print(spoken_languages)


        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in response_tmdb_query_details['production_countries']]


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_details = {
            "title"                 : title,
            "original_title"        : response_tmdb_query_details.get("original_title", ""),
            "budget"                : response_tmdb_query_details.get("budget", ""),
            "genres"                : genres,
            "language"              : response_tmdb_query_details.get("language", ""),
            "spoken_languages"      : spoken_languages,
            "homepage"              : response_tmdb_query_details.get("homepage", ""),
            "overview"              : response_tmdb_query_details.get("overview", ""),
            "popularity"            : response_tmdb_query_details.get("popularity", ""),
            "runtime"               : response_tmdb_query_details.get("runtime", ""),
            "revenue"               : response_tmdb_query_details.get("revenue", ""),
            "release_date"          : response_tmdb_query_details.get("release_date", ""),
            "vote_average"          : response_tmdb_query_details.get("vote_average", ""),
            "vote_count"            : response_tmdb_query_details.get("vote_count", ""),
            "production_countries"  : production_countries
        }
        tmdb_movies_list.append(movie_details)

        # Print out the title that was found
        print(f"Title Found: {title}")

    except:    
        # Print out the title that was not found
        print(f"Title Not Found: {title}")
        #break
        

#print(tmdb_movie_list)

Title Found: The Attachment Diaries
Title Not Found: What’s Love Got to Do With It?’ Probably a Lo
Title Found: You Can Live Forever
Title Found: A Tourist’s Guide to Love
Title Found: Other People’s Children
Title Found: One True Loves
Title Found: The Lost Weekend: A Love Story
Title Found: A Thousand and One
Title Found: Your Place or Mine
Title Found: Love in the Time of Fentanyl
Title Found: The Attachment Diaries
Title Not Found: What’s Love Got to Do With It?’ Probably a Lo
Title Found: You Can Live Forever
Title Found: A Tourist’s Guide to Love
Title Found: Other People’s Children
Title Found: One True Loves
Title Found: The Lost Weekend: A Love Story
Title Found: A Thousand and One
Title Found: Your Place or Mine
Title Found: Love in the Time of Fentanyl
Title Found: The Attachment Diaries
Title Not Found: What’s Love Got to Do With It?’ Probably a Lo
Title Found: You Can Live Forever
Title Found: A Tourist’s Guide to Love
Title Found: Other People’s Children
Title Found: One 

In [41]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for i in range(5):
    print(json.dumps(tmdb_movies_list[i], indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "language": "",
    "spoken_languages": [
        "Spanish"
    ],
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 1.802,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "You Can Live Forever",
    "original_title": "You Can Live Forever",
    "budget": 0,
    "genre

In [42]:
# Convert the results to a DataFrame
tmdb_df = pd.json_normalize(tmdb_movies_list)
tmdb_df

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.802,102,0,2021-10-07,3.000,4,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",,"[English, French]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,15055,2023-03-24,6.600,37,"[Canada, United States of America]"
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",,"[English, Vietnamese]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",23.710,96,0,2023-04-21,6.287,162,[United States of America]
3,Other People’s Children,Les Enfants des autres,0,"[Drama, Comedy]",,"[French, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",11.938,104,84178,2022-09-21,6.818,190,[France]
4,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.032,100,37820,2023-04-07,6.500,75,"[Czech Republic, United States of America]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",,"[English, Spanish]",,Emma and Jesse are living the perfect life tog...,14.032,100,37820,2023-04-07,6.500,75,"[Czech Republic, United States of America]"
176,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.002,95,77145,2023-04-13,6.000,2,[United States of America]
177,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",,"[English, Spanish, Portuguese]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.331,116,3395595,2023-03-31,6.871,89,[United States of America]
178,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,22.278,109,0,2023-02-10,6.279,711,[United States of America]


### Merge and Clean the Data for Export

In [43]:
# Merge the New York Times reviews and TMDB DataFrames on title
# DataFrames: nyt_reviews_df and tmdb_reviews_df
merged_df = pd.merge(tmdb_df, nyt_reviews_df, on='title', how='inner')
#deduplicated_df = merged_df.drop_duplicates(subset=['title'])
#deduplicated_df.reset_index(drop=True, inplace=True)
merged_df.head()


,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
4,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [44]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Create a list of characters to remove
characters_to_remove = ["'", "[", "]"]


# Loop through the list of columns to fix
for col in columns_to_fix:

    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)


    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, "")


# Display the fixed DataFrame
merged_df.head()


,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
3,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
4,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [45]:
# Drop "byline.person" column
print(merged_df.columns)
merged_df = merged_df.drop(columns=['byline.person'])
print(merged_df.columns)


Index(['title', 'original_title', 'budget', 'genres', 'language',
       'spoken_languages', 'homepage', 'overview', 'popularity', 'runtime',
       'revenue', 'release_date', 'vote_average', 'vote_count',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')
Index(['title', 'original_title', 'budget', 'genres', 'language',
       'spoken_languages', 'homepage', 'overview', 'popularity', 'runtime',
       'revenue', 'release_date', 'vote_average', 'vote_count',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline

In [46]:
# Delete duplicate rows and reset index
clean_df = merged_df.drop_duplicates()
clean_df.reset_index(drop=True, inplace=True)
clean_df

,title,original_title,budget,genres,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",,Spanish,,"Argentina, 1970s. A desperate young woman goes...",1.802,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",,"English, French",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",33.662,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,"Romance, Comedy",,"English, Vietnamese",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",23.710,96,...,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
3,Other People’s Children,Les Enfants des autres,0,"Drama, Comedy",,"French, English",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",11.938,104,...,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,None
4,One True Loves,One True Loves,0,"Romance, Comedy, Drama",,"English, Spanish",,Emma and Jesse are living the perfect life tog...,14.032,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,6.002,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
6,A Thousand and One,A Thousand and One,0,"Drama, Crime",,"English, Spanish, Portuguese",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,15.331,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
7,Your Place or Mine,Your Place or Mine,0,"Romance, Comedy",,English,https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,22.278,109,...,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,None
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,Documentary,,English,,An intimate portrait of a community fighting t...,0.896,85,...,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,None


In [47]:
# Export data to CSV without the index
clean_df.to_csv('clean_data.csv', index=False)